# Get and Preprocess data from Estonian Open Data Platform

**Index:**
1. [Data acquisition](#Data-acquisition)
2. [Investigation Data Structure](#Investigating-data-structure)
3. [Converting Data](#Converting-dataset)
4. [Data analysis](#Data-analysis)
5. [Get additional Data](#Get-additional-data)
6. [Export Final Data](#Export-final-values)

### Data acquisition 

Information on the COVID-19 open data can be found on the official Terviseamet website

[Koroonaviirus SARS-CoV-2 testide avaandmete kirjeldus](https://www.terviseamet.ee/et/koroonaviirus/avaandmed)

In the section Testide avaandmete andmestruktuuri kirjeldus > Avaandmete lingid is the URL to the JSON file in which all results are published.

In [1]:
url = r'https://opendata.digilugu.ee/opendata_covid19_test_results.json' # JSON document

With the [requests](https://requests.readthedocs.io/en/master/) library the file can be downloaded and read into variable ```d```.

In [2]:
import requests 

In [3]:
r = requests.get(url)
d = r.json()

The [list](https://www.w3schools.com/python/python_lists.asp) ```d``` now contains all the results from the COVID-19 testing. The amount of tests performed can be checked with ```len()```.

In [4]:
print(len(d))

62720


In the header data for the ```requests``` response is the ```Last-Modified``` date.

In [5]:
print(r.headers['Last-Modified'])

Sat, 09 May 2020 07:53:10 GMT


The derived results will be collected in the [dictionary](https://www.w3schools.com/python/python_dictionaries.asp) ```dc```.

In [6]:
dc = {}

In [7]:
#from datetime import datetime
from dateutil import tz
from dateutil.parser import parse
tzone = tz.gettz('Europe/Tallinn')

In [8]:
dc['totalTested'] = len(d)
dc['lastUpdate'] = parse(r.headers['Last-Modified']).astimezone(tzone).strftime('%d.%m.%Y %H:%M:%S')

### Investigating data structure

We can access the indivudal entries of the list ```d```. The first one with ```d[0]``` or the last one with ```d[-1]```. The list items are of type dictionary.

In [9]:
d[20]

{'id': '191b343ea1cbb67fbeb1f8440d80acfa1cc1df1c66e2921706935381ba67a765',
 'Gender': 'N',
 'AgeGroup': '35-39',
 'Country': 'Eesti',
 'County': 'Tartu maakond',
 'ResultValue': 'N',
 'StatisticsDate': '2020-03-11',
 'ResultTime': '2020-03-09T22:00:00+02:00',
 'AnalysisInsertTime': '2020-03-11T10:13:21+02:00'}

The fields in the resulting dictionary are described on the [official website](https://www.terviseamet.ee/et/koroonaviirus/avaandmed). The fields can be accessed individually:

In [10]:
d[20]['AgeGroup']

'35-39'

### Converting dataset

The [pandas](https://pandas.pydata.org/) library can be used to get the data in a more accessible way. It provides functions for further data analysis.

In [11]:
import pandas as pd

Convert the list with dictionary items to pandas dataframe. Display the first 5 rows.

In [12]:
df = pd.DataFrame(d)
df.head()

,id,Gender,AgeGroup,Country,County,ResultValue,StatisticsDate,ResultTime,AnalysisInsertTime
0,95013b64dd5ff18548a92eb5375d9c4a1881467390fed4...,M,10-14,Eesti,Tartu maakond,N,2020-03-10,2020-03-06T18:44:00+02:00,2020-03-10T16:01:55+02:00
1,71fab95aa66a3976b9d9f2868482192fc2bb77ac07d680...,M,5-9,Eesti,Tartu maakond,N,2020-03-10,2020-03-06T13:28:00+02:00,2020-03-10T16:05:53+02:00
2,e474cb8d21136013c9c90877592ee8d6b20d1bd72ef48a...,M,20-24,Eesti,Harju maakond,N,2020-03-10,2020-03-05T00:00:00+02:00,2020-03-10T15:53:52+02:00
3,86a33c6965a464b3c8b754795d99b3fccab5e8349827dc...,M,35-39,Eesti,Tartu maakond,N,2020-03-10,2020-03-05T00:00:00+02:00,2020-03-10T15:50:53+02:00
4,70fb213dfac6252426170b79224d399c6e613fbca07d54...,N,15-19,Eesti,Viljandi maakond,N,2020-03-10,2020-03-06T18:46:00+02:00,2020-03-10T15:59:21+02:00


#### Data cleaning

In [13]:
from aglearn import remap as rm # class has 

The ```ResultTime``` and ```AnalysisInsertTime``` are not of importance right now.

For the Maakonds an identification code is used, as the Counties have a long "Tartu maakond" and short "Tartumaa" way of spelling, which might get mixed up. The dictionary is saved in the custom ```aglearn``` library. 

Slightly adapt the Agegroup (leading zeros).

Transform the text in StatisticsDate into datetime objects.

In [14]:
df = df.drop(['ResultTime', 'AnalysisInsertTime', 'id'], axis=1)
df['MKOOD'] = df['County'].map(rm.MNIMI_MKOOD)
df['AgeGroup'] = df['AgeGroup'].map(rm.VANUSER_STR)
df['StatisticsDate'] = pd.to_datetime(df['StatisticsDate'])

In [15]:
df.head()

,Gender,AgeGroup,Country,County,ResultValue,StatisticsDate,MKOOD
0,M,10-14,Eesti,Tartu maakond,N,2020-03-10,0079
1,M,05-09,Eesti,Tartu maakond,N,2020-03-10,0079
2,M,20-24,Eesti,Harju maakond,N,2020-03-10,0037
3,M,35-39,Eesti,Tartu maakond,N,2020-03-10,0079
4,N,15-19,Eesti,Viljandi maakond,N,2020-03-10,0084


#### Export data

In [16]:
df.to_csv(r'covid_digilugu_cleaned.csv')

### Data analysis

In [17]:
dtFormat = '%d.%m.%Y'
dc['firstTest'] = df.StatisticsDate.min().strftime(dtFormat)
dc['lastTest'] = df.StatisticsDate.max().strftime(dtFormat)

#### Positive vs Negative

The function ```value_counts()``` can be used to summarize the respective columns.

In [18]:
df.ResultValue.value_counts()

N    60987
P     1733
Name: ResultValue, dtype: int64

In [19]:
dc['totalPositive'] = int(df.ResultValue.value_counts()['P'])
dc['totalNegative'] = int(df.ResultValue.value_counts()['N'])

In [20]:
dc['percPositive'] = round(dc['totalPositive']/dc['totalTested'],4)

#### Values Last day

In [22]:
from datetime import datetime as dt

In [23]:
res = df[df.StatisticsDate == dt.strptime(dc['lastTest'],dtFormat)].ResultValue.value_counts()
res

N    946
P      8
Name: ResultValue, dtype: int64

In [24]:
if 'P' in res: # in case there are no positive results :) 
    dc['prevDayConfirmed'] = int(res['P'])
else:
    dc['prevDayConfirmed'] = 0
dc['prevDayTests'] = int(res['N'] + dc['prevDayConfirmed'])

#### Timeseries Estonia

For further statistics a one-hot encoding has to be applied on the dataset. The result is joined with the dataframe.

In [25]:
df1h = pd.get_dummies(df[['ResultValue']], prefix=['Results'])
df1h = df1h.rename(columns={'Results_N' : 'negativeTests', 'Results_P' : 'confirmedCases'})
df1h = df.join(df1h)
df1h[-10:]

,Gender,AgeGroup,Country,County,ResultValue,StatisticsDate,MKOOD,negativeTests,confirmedCases
62710,M,30-34,Eesti,Harju maakond,N,2020-05-08,0037,1,0
62711,N,20-24,Eesti,Järva maakond,N,2020-05-08,0052,1,0
62712,N,40-44,Eesti,Harju maakond,N,2020-05-08,0037,1,0
62713,N,00-04,Eesti,Järva maakond,N,2020-05-08,0052,1,0
62714,M,35-39,Eesti,Rapla maakond,N,2020-05-08,0071,1,0
62715,M,35-39,Eesti,Tartu maakond,N,2020-05-08,0079,1,0
62716,M,35-39,Eesti,Rapla maakond,N,2020-05-08,0071,1,0
62717,N,50-54,Eesti,Lääne maakond,N,2020-05-08,0056,1,0
62718,N,75-79,Eesti,Lääne maakond,N,2020-05-08,0056,1,0
62719,N,45-49,Eesti,Harju maakond,N,2020-05-08,0037,1,0


In [26]:
newDateRange = pd.date_range(start=dt.strptime(dc['firstTest'], dtFormat), end=dt.strptime(dc['lastTest'], dtFormat), freq='1D')

In [27]:
dfts = df1h.groupby(['StatisticsDate']).sum()
dfts['testsPerDay'] = df1h.groupby(['StatisticsDate']).count().values[:,1]
dfts = dfts.reindex(newDateRange)
dfts = dfts.fillna(0)
dfts

,negativeTests,confirmedCases,testsPerDay
2020-02-05,1.0,0.0,1.0
2020-02-06,1.0,0.0,1.0
2020-02-07,0.0,0.0,0.0
2020-02-08,0.0,0.0,0.0
2020-02-09,0.0,0.0,0.0
...,...,...,...
2020-05-04,1641.0,8.0,1649.0
2020-05-05,1536.0,2.0,1538.0
2020-05-06,1540.0,7.0,1547.0
2020-05-07,1273.0,5.0,1278.0


Cumulative Sums

In [28]:
dfts['cumulativeNegative'] = dfts['negativeTests'].cumsum()
dfts['cumulativePositive'] = dfts['confirmedCases'].cumsum()
dfts['testsPerformed'] = dfts['testsPerDay'].cumsum()
dfts

,negativeTests,confirmedCases,testsPerDay,cumulativeNegative,cumulativePositive,testsPerformed
2020-02-05,1.0,0.0,1.0,1.0,0.0,1.0
2020-02-06,1.0,0.0,1.0,2.0,0.0,2.0
2020-02-07,0.0,0.0,0.0,2.0,0.0,2.0
2020-02-08,0.0,0.0,0.0,2.0,0.0,2.0
2020-02-09,0.0,0.0,0.0,2.0,0.0,2.0
...,...,...,...,...,...,...
2020-05-04,1641.0,8.0,1649.0,55692.0,1711.0,57403.0
2020-05-05,1536.0,2.0,1538.0,57228.0,1713.0,58941.0
2020-05-06,1540.0,7.0,1547.0,58768.0,1720.0,60488.0
2020-05-07,1273.0,5.0,1278.0,60041.0,1725.0,61766.0


Percentages 

In [29]:
dfts['positiveTestsPerc'] = (dfts['confirmedCases' ]/dfts['testsPerDay']).round(4)
dfts['positiveTestsPercCum'] = (dfts['cumulativePositive' ]/dfts['testsPerformed']).round(4)
dfts.loc[dfts.index[-1], 'lastFeature'] = 1
dfts = dfts.reset_index().rename(columns={'index':'StatisticsDate'})
dfts

,StatisticsDate,negativeTests,confirmedCases,testsPerDay,cumulativeNegative,cumulativePositive,testsPerformed,positiveTestsPerc,positiveTestsPercCum,lastFeature
0,2020-02-05,1.0,0.0,1.0,1.0,0.0,1.0,0.0000,0.0000,NaN
1,2020-02-06,1.0,0.0,1.0,2.0,0.0,2.0,0.0000,0.0000,NaN
2,2020-02-07,0.0,0.0,0.0,2.0,0.0,2.0,NaN,0.0000,NaN
3,2020-02-08,0.0,0.0,0.0,2.0,0.0,2.0,NaN,0.0000,NaN
4,2020-02-09,0.0,0.0,0.0,2.0,0.0,2.0,NaN,0.0000,NaN
...,...,...,...,...,...,...,...,...,...,...
89,2020-05-04,1641.0,8.0,1649.0,55692.0,1711.0,57403.0,0.0049,0.0298,NaN
90,2020-05-05,1536.0,2.0,1538.0,57228.0,1713.0,58941.0,0.0013,0.0291,NaN
91,2020-05-06,1540.0,7.0,1547.0,58768.0,1720.0,60488.0,0.0045,0.0284,NaN
92,2020-05-07,1273.0,5.0,1278.0,60041.0,1725.0,61766.0,0.0039,0.0279,NaN


#### Timeseries Maakond

In [30]:
counties = list(df1h['County'].unique())
counties.remove('')
counties

['Tartu maakond',
 'Harju maakond',
 'Viljandi maakond',
 'Valga maakond',
 'Võru maakond',
 'Pärnu maakond',
 'Jõgeva maakond',
 'Lääne maakond',
 'Saare maakond',
 'Lääne-Viru maakond',
 'Põlva maakond',
 'Ida-Viru maakond',
 'Rapla maakond',
 'Hiiu maakond',
 'Järva maakond']

In [31]:
i = 0
for county in counties:
    dftsm0 = df1h.loc[df1h['County'] == county] # select a subset 
    dftsm = dftsm0.groupby(['StatisticsDate']).sum() # group by date and county
    dftsm['testsPerDay'] = dftsm0.groupby(['StatisticsDate']).count().values[:,1]
    dftsm = dftsm.reindex(newDateRange)
    dftsm = dftsm.fillna(0)
    dftsm['cumulativeNegative'] = dftsm['negativeTests'].cumsum()
    dftsm['cumulativePositive'] = dftsm['confirmedCases'].cumsum()
    dftsm['testsPerformed'] = dftsm['testsPerDay'].cumsum()
    dftsm.loc[dftsm.index[-1], 'lastFeature'] = 1
    dftsm['County'] = county
    #dftsm['MKOOD'] = rm.MNIMI_MKOOD[county]
    if i == 0:
        dftsm_all = dftsm
        i += 1
    else:
        dftsm_all = dftsm_all.append(dftsm)
dftsm_all['positiveTestsPerc'] = (dftsm_all['confirmedCases' ]/dftsm_all['testsPerDay']).round(4)
dftsm_all['positiveTestsPercCum'] = (dftsm_all['cumulativePositive' ]/dftsm_all['testsPerformed']).round(4)
dftsm_all = dftsm_all.reset_index().rename(columns={'index':'StatisticsDate'})
dftsm_all

,StatisticsDate,negativeTests,confirmedCases,testsPerDay,cumulativeNegative,cumulativePositive,testsPerformed,lastFeature,County,positiveTestsPerc,positiveTestsPercCum
0,2020-02-05,0.0,0.0,0.0,0.0,0.0,0.0,NaN,Tartu maakond,NaN,NaN
1,2020-02-06,0.0,0.0,0.0,0.0,0.0,0.0,NaN,Tartu maakond,NaN,NaN
2,2020-02-07,0.0,0.0,0.0,0.0,0.0,0.0,NaN,Tartu maakond,NaN,NaN
3,2020-02-08,0.0,0.0,0.0,0.0,0.0,0.0,NaN,Tartu maakond,NaN,NaN
4,2020-02-09,0.0,0.0,0.0,0.0,0.0,0.0,NaN,Tartu maakond,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
1405,2020-05-04,75.0,0.0,75.0,1548.0,13.0,1561.0,NaN,Järva maakond,0.0,0.0083
1406,2020-05-05,25.0,0.0,25.0,1573.0,13.0,1586.0,NaN,Järva maakond,0.0,0.0082
1407,2020-05-06,77.0,0.0,77.0,1650.0,13.0,1663.0,NaN,Järva maakond,0.0,0.0078
1408,2020-05-07,26.0,0.0,26.0,1676.0,13.0,1689.0,NaN,Järva maakond,0.0,0.0077


### Get additional data

#### Webscraping Terviseamet

Some data is not available in the open data, but can be acquired from the website. 
The library [beautifulsoup](https://www.crummy.com/software/BeautifulSoup/bs4/doc/) can be utilized to scrape the data.

(Hint: In case BeautifulSoup is not installed in the Python Envrionment, consult this [arcticle](https://pro.arcgis.com/en/pro-app/arcpy/get-started/what-is-conda.htm) on how to clone the default environment and Add Packages.)

In [32]:
from bs4 import BeautifulSoup
import re

In [33]:
url2 = r'https://www.terviseamet.ee/et/koroonaviirus/koroonakaart'

Download and parse the website.

In [34]:
r2 = requests.get(url2)
soup = BeautifulSoup(r2.text, features="html.parser")

Values that need to be extracted and their translation.

In [37]:
toScrp = {"KINNITATUD SURMAD": "deceasedCases",
        "HAIGLAST VÄLJAKIRJUTATUD": "recoveredCases",
        "HAIGLARAVIL": "hospitalisedCases" }

In [38]:
for scrp in toScrp.keys():
    res = soup.find(text=re.compile(scrp)) # looks for the string
    exstr = res.find_parent('div').text #extracts the text from the parent container
    dc[toScrp[scrp]] = int(re.findall(r"\n([0-9]{1,4})\n", exstr)[0]) # extracts the case number with regular expression
    print("{} ({}): {}".format(scrp, toScrp[scrp], dc[toScrp[scrp]]))

KINNITATUD SURMAD (deceasedCases): 60
HAIGLAST VÄLJAKIRJUTATUD (hospitalisedCases): 276
HAIGLARAVIL (recoveredCases): 48


### Export Final Data

In the dictionary ```dc``` all the derived statistics are stored.

In [41]:
dc

{'totalTested': 62720,
 'lastUpdate': '09.05.2020 10:53:10',
 'firstTest': '05.02.2020',
 'lastTest': '08.05.2020',
 'totalPositive': 1733,
 'totalNegative': 60987,
 'percPositive': 0.0276,
 'prevDayConfirmed': 8,
 'prevDayTests': 954,
 'deceasedCases': 60,
 'hospitalisedCases': 276,
 'recoveredCases': 48}

In [42]:
import json

In [43]:
with open(r'data/cov_stats_eesti.json', 'w') as f:
    json.dump(dc, f, indent=4)

The dataframe ```dfts``` contains the timeseries for whole Estonia.

In [44]:
dfts

,StatisticsDate,negativeTests,confirmedCases,testsPerDay,cumulativeNegative,cumulativePositive,testsPerformed,positiveTestsPerc,positiveTestsPercCum,lastFeature
0,2020-02-05,1.0,0.0,1.0,1.0,0.0,1.0,0.0000,0.0000,NaN
1,2020-02-06,1.0,0.0,1.0,2.0,0.0,2.0,0.0000,0.0000,NaN
2,2020-02-07,0.0,0.0,0.0,2.0,0.0,2.0,NaN,0.0000,NaN
3,2020-02-08,0.0,0.0,0.0,2.0,0.0,2.0,NaN,0.0000,NaN
4,2020-02-09,0.0,0.0,0.0,2.0,0.0,2.0,NaN,0.0000,NaN
...,...,...,...,...,...,...,...,...,...,...
89,2020-05-04,1641.0,8.0,1649.0,55692.0,1711.0,57403.0,0.0049,0.0298,NaN
90,2020-05-05,1536.0,2.0,1538.0,57228.0,1713.0,58941.0,0.0013,0.0291,NaN
91,2020-05-06,1540.0,7.0,1547.0,58768.0,1720.0,60488.0,0.0045,0.0284,NaN
92,2020-05-07,1273.0,5.0,1278.0,60041.0,1725.0,61766.0,0.0039,0.0279,NaN


In [45]:
#dfts = dfts.reset_index()
#dfts = dfts.rename(column={})
dfts.to_csv(r'data/cov_ts_eesti.csv', index=False)

The dataframe ```dftsm_all``` contains the timeseries for each County.

In [46]:
dftsm_all

,StatisticsDate,negativeTests,confirmedCases,testsPerDay,cumulativeNegative,cumulativePositive,testsPerformed,lastFeature,County,positiveTestsPerc,positiveTestsPercCum
0,2020-02-05,0.0,0.0,0.0,0.0,0.0,0.0,NaN,Tartu maakond,NaN,NaN
1,2020-02-06,0.0,0.0,0.0,0.0,0.0,0.0,NaN,Tartu maakond,NaN,NaN
2,2020-02-07,0.0,0.0,0.0,0.0,0.0,0.0,NaN,Tartu maakond,NaN,NaN
3,2020-02-08,0.0,0.0,0.0,0.0,0.0,0.0,NaN,Tartu maakond,NaN,NaN
4,2020-02-09,0.0,0.0,0.0,0.0,0.0,0.0,NaN,Tartu maakond,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
1405,2020-05-04,75.0,0.0,75.0,1548.0,13.0,1561.0,NaN,Järva maakond,0.0,0.0083
1406,2020-05-05,25.0,0.0,25.0,1573.0,13.0,1586.0,NaN,Järva maakond,0.0,0.0082
1407,2020-05-06,77.0,0.0,77.0,1650.0,13.0,1663.0,NaN,Järva maakond,0.0,0.0078
1408,2020-05-07,26.0,0.0,26.0,1676.0,13.0,1689.0,NaN,Järva maakond,0.0,0.0077


In [47]:
dftsm_all.to_csv(r'data/ts_maakond.csv', index=False)